In [15]:
import re
from pathlib import Path
import numpy as np
import pandas as pd

In [16]:
# Folder containing your M***.xlsx files:
INPUT_DIR = Path(r"Z:\UriMons\Fiber Photometry\GCaMP\adBNST_eEPA_GCaMP8s\Event_Detection\Threshold_Prominence_2\KO\5minBins")
# Where to save the output file:
OUTPUT_PATH = INPUT_DIR / "Events_summary.xlsx"

In [17]:
def extract_mouse_id(path: Path) -> str | None:
    m = re.match(r"^(M\d+)", path.stem, flags=re.IGNORECASE)
    return m.group(1).upper() if m else None

def read_bin_summary(file_path: Path) -> pd.DataFrame:
    """
    Reads the 'Bin_summary' sheet, returns standardized columns:
    ['bin', 'event_count', 'avg_peak_amp', 'total_area']
    """
    try:
        df = pd.read_excel(file_path, sheet_name="Bin_summary", engine="openpyxl")
    except Exception as e:
        raise ValueError(f"Couldn't read 'Bin_summary' in {file_path.name}: {e}")

    # Normalize headers
    df.columns = df.columns.str.strip().str.lower()

    # Flexible column mapping
    def find_col(patterns):
        for name in df.columns:
            for pat in patterns:
                if re.fullmatch(pat, name):
                    return name
        return None

    col_bin  = find_col([r"bin"])
    col_ev   = find_col([r"event[_\s]*count"])
    col_amp  = find_col([r"avg[_\s]*peak[_\s]*amp"])
    col_area = find_col([r"total[_\s]*area"])

    missing = [n for n, c in {
        "bin": col_bin, "event_count": col_ev, "avg_peak_amp": col_amp, "total_area": col_area
    }.items() if c is None]
    if missing:
        raise ValueError(f"{file_path.name}: missing columns {missing}. Found: {list(df.columns)}")

    out = df[[col_bin, col_ev, col_amp, col_area]].copy()
    out.columns = ["bin", "event_count", "avg_peak_amp", "total_area"]
    return out

def add_stats_and_filtered_block(rows: pd.DataFrame, mouse_cols: list[str]) -> pd.DataFrame:
    """
    Layout:
      bin | <mouse cols> | "" | avg_raw | sd_raw | sem_raw | avg_plus_2sem_raw | avg_minus_2sem_raw
          | "" | <mouse cols (filtered)>
          | "" | avg | sd | sem | avg_plus_2sem | avg_minus_2sem    # recomputed from filtered
    """
    out = rows[["bin"] + mouse_cols].copy()

    # ---- RAW STATS (used for outlier thresholds) ----
    values_raw = out[mouse_cols]
    n_raw = values_raw.count(axis=1)
    mean_raw = values_raw.mean(axis=1, skipna=True)
    sd_raw = values_raw.std(axis=1, ddof=1)
    sem_raw = sd_raw / np.sqrt(n_raw.replace(0, np.nan))
    upper_raw = mean_raw + 3 * sem_raw
    lower_raw = mean_raw - 3 * sem_raw

    # Add raw stats after a spacer
    out[""] = ""  # spacer
    out["avg_raw"] = mean_raw
    out["sd_raw"] = sd_raw
    out["sem_raw"] = sem_raw
    out["avg +3sem_raw"] = upper_raw
    out["avg -3sem_raw"] = lower_raw

    # ---- FILTERED MOUSE COLUMNS (based on raw thresholds) ----
    out["  "] = ""  # spacer after raw stats
    filtered_block = pd.DataFrame(index=out.index)
    for col in mouse_cols:
        filtered_block[f"{col} (filtered)"] = values_raw[col].where(
            (values_raw[col] >= lower_raw) & (values_raw[col] <= upper_raw)
        )
    out = pd.concat([out, filtered_block], axis=1)

    # ---- RECOMPUTE STATS *AFTER* FILTERING ----
    out["   "] = ""  # spacer before filtered stats
    values_filt = filtered_block  # already aligned
    n_f = values_filt.count(axis=1)
    mean_f = values_filt.mean(axis=1, skipna=True)
    sd_f = values_filt.std(axis=1, ddof=1)
    sem_f = sd_f / np.sqrt(n_f.replace(0, np.nan))
    upper_f = mean_f + 3 * sem_f
    lower_f = mean_f - 3 * sem_f

    # These are the "remade" stats you asked for
    out["avg"] = mean_f
    out["sd"] = sd_f
    out["sem"] = sem_f
    out["avg +3sem"] = upper_f
    out["avg -3sem"] = lower_f

    return out

In [18]:
def main():
    # Collect files starting with M and digits
    candidates = [p for p in INPUT_DIR.glob("M*.xlsx") if p.is_file()]
    items = []
    for p in candidates:
        mid = extract_mouse_id(p)
        if mid:
            items.append((p, mid))
    if not items:
        raise SystemExit(f"No Excel files starting with 'M' and digits found in: {INPUT_DIR}")

    # Sort by numeric part
    items.sort(key=lambda it: int(re.search(r"M(\d+)", it[1]).group(1)))

    merged = {"event_count": None, "avg_peak_amp": None, "total_area": None}
    mouse_ids = []

    for file_path, mouse_id in items:
        df = read_bin_summary(file_path)
        parts = {
            "event_count": df[["bin", "event_count"]].rename(columns={"event_count": mouse_id}),
            "avg_peak_amp": df[["bin", "avg_peak_amp"]].rename(columns={"avg_peak_amp": mouse_id}),
            "total_area":   df[["bin", "total_area"]].rename(columns={"total_area": mouse_id}),
        }
        for key, piece in parts.items():
            merged[key] = piece if merged[key] is None else merged[key].merge(piece, on="bin", how="outer")
        mouse_ids.append(mouse_id)

    def sort_by_bin(df):
        b = pd.to_numeric(df["bin"], errors="coerce")
        if b.notna().any():
            return df.assign(_bn=b).sort_values(by=["_bn", "bin"], kind="mergesort").drop(columns="_bn")
        return df

    for k in merged:
        merged[k] = sort_by_bin(merged[k])

    # Build final sheets
    sheet_events = add_stats_and_filtered_block(merged["event_count"], mouse_ids)
    sheet_amp    = add_stats_and_filtered_block(merged["avg_peak_amp"], mouse_ids)
    sheet_area   = add_stats_and_filtered_block(merged["total_area"],   mouse_ids)

    with pd.ExcelWriter(OUTPUT_PATH, engine="openpyxl") as writer:
        sheet_events.to_excel(writer, index=False, sheet_name="Events_summary")
        sheet_amp.to_excel(writer,    index=False, sheet_name="Avg_peak_amp_summary")
        sheet_area.to_excel(writer,   index=False, sheet_name="Total_area_summary")

    print(f"Saved: {OUTPUT_PATH}")

if __name__ == "__main__":
    main()

Saved: Z:\UriMons\Fiber Photometry\GCaMP\adBNST_eEPA_GCaMP8s\Event_Detection\Threshold_Prominence_2\KO\5minBins\Events_summary.xlsx
